In [ ]:
import pandas as pd
import numpy as np
import os
import itertools
import matplotlib.pyplot as plt
from matplotlib import rcParams, gridspec
from pandas.api.types import CategoricalDtype

import sklearn
from sklearn import model_selection
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [ ]:
import gc
gc.collect()

In [ ]:
os.chdir("/Users/kelsey.huntzberry/Documents/Opioid_Research/Python")
print(os.getcwd())

In [ ]:
# Read in Treatment Episode Data Set data
teds1517 = pd.read_csv('/Users/kelsey.huntzberry/Documents/Opioid_Research/Data/TEDS/Raw_Data/tedsa_puf_2015_2017.csv')
teds14 = pd.read_csv('/Users/kelsey.huntzberry/Documents/Opioid_Research/Data/TEDS/Raw_Data/tedsa_puf_2014.csv')
teds13 = pd.read_csv('/Users/kelsey.huntzberry/Documents/Opioid_Research/Data/TEDS/Raw_Data/tedsa_puf_2013.csv')

In [ ]:
# Change column names so data can be appended
teds1517 = teds1517.drop(columns = "FREQ_ATND_SELF_HELP")
teds13 = teds13.drop(columns = "PMSA")
teds1517.rename(columns = {"CBSA2010":"CBSA",
                          "ADMYR":"YEAR",
                          "SERVICES":"SERVSETA"}, inplace = True)

In [ ]:
# Subset the data to just the 0/1 drug flag variables
flags = teds1517.filter(regex='FLG$', axis = 1)

In [ ]:
# Sum the flag variables to calculate the number of drugs recorded for each individual
NUMSUBS = flags.sum(axis=1)

In [ ]:
# Concatenate flag variables back into the 2015-17 data
teds_wflgs = pd.concat([teds1517, NUMSUBS], axis = 1)
teds_wflgs.rename(columns={0:'NUMSUBS'}, inplace = True)

In [ ]:
# Reorder the 2015-17 columns for appending
col_names = list(teds14.columns)
teds_reorder = teds_wflgs[col_names]

In [ ]:
# Append all years together
teds_all_temp = teds_reorder.append(teds14)
teds_all = teds_all_temp.append(teds13)

In [ ]:
# Select subset of columns
teds_sm = teds_all.loc[:,['CASEID','YEAR','AGE','GENDER','RACE','ETHNIC','MARSTAT','EDUC','EMPLOY','VET','LIVARAG',
                         'ARRESTS','DIVISION','SERVSETA','PSOURCE','NOPRIOR','SUB1','FRSTUSE1','NUMSUBS','PSYPROB']]

In [ ]:
# Remove rows where the first substance was "None"
teds_sm_temp = teds_sm[teds_sm.SUB1 != 1]
# Remove rows where number of prior treatments is NA (target variable)
teds_sm1 = teds_sm_temp[teds_sm_temp.NOPRIOR != -9]

In [ ]:
# Recode age group variable
def age_groups(series):
    # 12-17 years
    if series == 1 or series == 2:
        return '12-17 years'
    # 18-29 years
    elif series == 3 or series == 4 or series == 5:
        return '18-29 years'
    # 30-39 years
    elif series == 6 or series == 7:
        return '30-39 years'
    # 40-49 years
    elif series == 8 or series == 9:
        return '40-49 years'
    # 50-64 years
    elif series == 10 or series == 11:
        return '50-64 years'
    # 65+ years
    elif series == 12:
        return '65+ years'
    
teds_sm1.loc[:, 'age_group'] = teds_sm1.AGE.apply(age_groups)

# Change variable to an ordered factor
teds_sm1.loc[:, 'age_group'] = pd.Categorical(teds_sm1['age_group'], categories = ['12-17 years', '18-29 years', '30-39 years',
                                                                                   '40-49 years', '50-64 years', '65+ years'], ordered = True)

# Change variable to an ordered factor with values as numbers
labels, unique = pd.factorize(teds_sm1.loc[:, 'age_group'], sort = True)
teds_sm1.loc[:, 'age_group'] = labels

In [ ]:
# Recode primary substance variable
def sub(series):
    if series == 2:
        return 'Alcohol'
    elif series == 3:
        return 'Cocaine/Crack'
    elif series == 4:
        return 'Marijuana/Hashish'
    elif series == 5:
        return 'Heroin'
    elif series == 6 or series == 7:
        return 'Opioids or Methadone'
    elif series == 10:
        return 'Methamphetamine'
    elif series == 11 or series == 12:
        return 'Other Amphetamines or Stimulants'
    elif series == 13 or series == 14:
        return 'Benzodiazepines or Tranquilizers'
    elif series == 8 or series == 9:
        return 'PCP or Other Hallucinogens'
    elif series >= 15 & series <= 20:
        return 'Other'

teds_sm1.loc[:, 'drug'] = teds_sm1.SUB1.apply(sub)

In [ ]:
# Recode gender variable
def gen_rc(series):
    if series == 1:
        return 'Male'
    elif series == 2:
        return 'Female'
    
teds_sm1.loc[:, 'gender'] = teds_sm1.GENDER.apply(gen_rc)

In [ ]:
# Record race variable
def race_rc(series):
    if series == 1:
        return 'Alaska Native'
    elif series == 2:
        return 'American Indian'
    elif series == 3 or series == 9:
        return 'Hawaii, Pacific Islander'
    elif series == 4:
        return 'Black'
    elif series == 5:
        return 'White'
    elif series == 6:
        return 'Asian'
    elif series == 7:
        return 'Other race'
    elif series == 8:
        return 'Two or more races'
    
teds_sm1.loc[:, 'race'] = teds_sm1.RACE.apply(race_rc)

In [ ]:
# Recode ethnicity variable
def ethnic_rc(series):
    if (series >= 1 or series <= 3) or series == 5:
        return 'Hispanic'
    elif series == 4:
        return 'Non-Hispanic'
    
teds_sm1.loc[:, 'ethnic'] = teds_sm1.ETHNIC.apply(ethnic_rc)

In [ ]:
# Recode service setting variable
def servseta_rc(series):
    if series == 1 or series == 2:
        return 'Detox'
    elif series >= 3 and series <= 5:
        return 'Rehab/Residential'
    elif series >= 6 and series <= 8:
        return 'Ambulatory'

teds_sm1.loc[:, 'servseta'] = teds_sm1.SERVSETA.apply(servseta_rc)

In [ ]:
# Recode marital status variable
def marstat_rc(series):
    if series == 1:
        return 'Never Married'
    elif series == 2:
        return 'Married'
    elif series == 3:
        return 'Separated'
    elif series == 4:
        return 'Divorced or Widowed'

teds_sm1.loc[:, 'marstat'] = teds_sm1.MARSTAT.apply(marstat_rc)

In [ ]:
# Recode employment status variable
def employ_rc(series):
    if series == 1:
        return 'Full-time'
    elif series == 2:
        return 'Part-time'
    elif series == 3:
        return 'Unemployed'
    elif series == 4:
        return 'Not in the labor force'
    
teds_sm1.loc[:, 'employ'] = teds_sm1.EMPLOY.apply(employ_rc)

In [ ]:
# Recode veteran variable
def vet_rc(series):
    if series == 1:
        return 'Veteran'
    elif series == 2:
        return 'Not a Veteran'
    
teds_sm1.loc[:, 'vet'] = teds_sm1.VET.apply(vet_rc)

In [ ]:
# Recode living arrangement variable
def livarag_rc(series):
    if series == 1:
        return 'Homeless'
    elif series == 2:
        return 'Dependent Living'
    elif series == 3:
        return 'Independent Living'

teds_sm1.loc[:, 'livarag'] = teds_sm1.LIVARAG.apply(livarag_rc)

In [ ]:
# Recode arrests variable
def arrests_rc(series):
    if series == 0:
        return 'None'
    elif series == 1:
        return 'Once'
    elif series == 2:
        return '2 or more times'
    
teds_sm1.loc[:, 'arrests'] = teds_sm1.ARRESTS.apply(arrests_rc)

# Change variable to an ordered factor variable
teds_sm1.loc[:, 'arrests'] = pd.Categorical(teds_sm1['arrests'], categories = ['None', 'Once',
                                                                              '2 or more times'],
                                           ordered = True)

# Change variable to an ordered factor with values as numbers
labels, unique = pd.factorize(teds_sm1.loc[:, 'arrests'], sort = True)
teds_sm1.loc[:, 'arrests'] = labels

In [ ]:
# Recode division variable
def division_rc(series):
    if series == 0:
        return 'US Territories'
    elif series == 1:
        return 'New England'
    elif series == 2:
        return 'Mid Atlantic'
    elif series == 3:
        return 'East North Central'
    elif series == 4:
        return 'West North Central'
    elif series == 5:
        return 'South Atlantic'
    elif series == 6:
        return 'East South Central'
    elif series == 7:
        return 'West South Central'
    elif series == 8:
        return 'Mountain'
    elif series == 9:
        return 'Pacific'
    
teds_sm1.loc[:, 'division'] = teds_sm1.DIVISION.apply(division_rc)

In [ ]:
# Recode referral source variable
def psource_rc(series):
    if series == 1:
        return 'Self-referral'
    elif series == 2:
        return 'Alcohol or Drug Care Professional'
    elif series == 3:
        return 'Other Health Care Professional'
    elif series == 4:
        return 'School Referral'
    elif series == 5:
        return 'Employer Referral'
    elif series == 6:
        return 'Community Referral'
    elif series == 7:
        return 'Court Referral'
    
teds_sm1.loc[:, 'psource'] = teds_sm1.PSOURCE.apply(psource_rc)

In [ ]:
# Recode number of prior treatment encounters
def noprior_rc(series):
    if series == 0:
        return 0
    elif series >= 1:
        return 1
    
teds_sm1.loc[:, 'noprior'] = teds_sm1.NOPRIOR.apply(noprior_rc)

In [ ]:
# Recode year variable
def year_rc(series):
    if series == 2013:
        return '2013'
    elif series == 2014:
        return '2014'
    elif series == 2015:
        return '2015'
    elif series == 2016:
        return '2016'
    elif series == 2017:
        return '2017'
    
teds_sm1.loc[:, 'year'] = teds_sm1.YEAR.apply(year_rc)


# Change year to an ordered factor
teds_sm1.loc[:, 'year'] = pd.Categorical(teds_sm1['year'], categories = ['2013', '2014', '2015', '2016', '2017'],
                                           ordered = True)

# Convert year to factor with numeric value
labels, unique = pd.factorize(teds_sm1.loc[:, 'year'], sort = True)
teds_sm1.loc[:, 'year'] = labels

In [ ]:
# Recode first use variable
def frstuse_rc(series):
    if series == 1:
        return '11 years or under'
    elif series == 2:
        return '12-14 years'
    elif series == 3:
        return '15-17 years'
    elif series == 4:
        return '18-20 years'
    elif series == 5:
        return '21-24 years'
    elif series == 6:
        return '25-29 years'
    elif series == 7:
        return '30 years or older'
    
teds_sm1.loc[:, 'frstuse'] = teds_sm1.FRSTUSE1.apply(frstuse_rc)

# Change first use into an ordered factor
teds_sm1.loc[:, 'frstuse'] = pd.Categorical(teds_sm1['frstuse'], categories = ['11 years or under', '12-14 years', '15-17 years',
                                                                               '18-20 years', '21-24 years', '25-29 years',
                                                                               '30 years or older'], ordered = True)

# Convert year to factor with numeric value
labels, unique = pd.factorize(teds_sm1.loc[:, 'frstuse'], sort = True)
teds_sm1.loc[:, 'frstuse'] = labels

In [ ]:
# Recode mental illness variable
def psyprob_rc(series):
    if series == 1:
        return 'Has mental illness'
    elif series == 2:
        return 'Does not have mental illness'
    
teds_sm1.loc[:, 'psyprob'] = teds_sm1.PSYPROB.apply(psyprob_rc)

In [ ]:
# Recode number of substances variable
def numsubs_rc(series):
    if series == 0:
        return 'Zero substances'
    elif series == 1:
        return 'One substance'
    elif series == 2:
        return 'Two substances'
    elif series == 3:
        return 'Three substances'
    
teds_sm1.loc[:, 'numsubs'] = teds_sm1.NUMSUBS.apply(numsubs_rc)

# Change first use into an ordered factor
teds_sm1.loc[:, 'numsubs'] = pd.Categorical(teds_sm1['numsubs'], categories = ['Zero substances', 'One substance',
                                                                               'Two substances', "Three substances"], ordered = True)

# Convert year to factor with numeric value
labels, unique = pd.factorize(teds_sm1.loc[:, 'numsubs'], sort = True)
teds_sm1.loc[:, 'numsubs'] = labels

In [ ]:
# Subset to fewer variables, mostly dropping those with many missing values
teds_clean = teds_sm1.drop(['CASEID', 'YEAR', 'AGE', 'GENDER', 'RACE', 'ETHNIC', 'MARSTAT',
                            'EDUC', 'EMPLOY', 'VET', 'LIVARAG', 'ARRESTS', 'DIVISION', 'SERVSETA',
                            'PSOURCE', 'NOPRIOR', 'SUB1', 'FRSTUSE1', 'NUMSUBS', 'PSYPROB'], axis = 1)

In [ ]:
# Create dummy variables for unordered categorical variables
teds_clean = pd.get_dummies(teds_clean, columns=['drug', 'gender', 'race', 'ethnic', 'servseta', 'marstat', 'employ', 'livarag', 
                                             'division', 'psource', 'psyprob', 'vet'])

In [ ]:
# Subset data to just 2017
# Attempted analyses with 2015-17 as well but the results were about the same as without the extra years
teds2017 = teds_clean.query('year == 4')

In [ ]:
# Drop response variable and year since only 2017 will be used in final modeling
data = teds2017.drop(columns = ['noprior', 'year'])
# Create data frame with just the response variable
response = teds2017.loc[:,['noprior']]

In [ ]:
# Impute missing data witn mode
my_imputer = SimpleImputer()
data_imputed = pd.DataFrame(my_imputer.fit_transform(data))
data_imputed.columns = data.columns

In [ ]:
# Change response and predictor data frames to numpy arrays
data_imp_np = np.array(data_imputed)
response_np = np.array(response)

In [ ]:
# Create holdout data set and keep remaining 80% in one data frame
# Used stratefied random sampling because there was class imbalance
sss = StratifiedShuffleSplit(n_splits = 2, test_size=0.2, random_state=0)

sss.get_n_splits(data_imp_np, response_np)

for train_index, test_index in sss.split(data_imp_np, response_np):
    x_train_temp, x_test = data_imp_np[train_index], data_imp_np[test_index]
    y_train_temp, y_test = response_np[train_index], response_np[test_index]

# Split the remaining data into a training and validation data set (50% and 30% respectively)
sss_valid = StratifiedShuffleSplit(n_splits = 2, test_size = 0.3, random_state = 10)  
    
for train_index, test_index in sss_valid.split(x_train_temp, y_train_temp):
    x_train, x_validation = x_train_temp[train_index], x_train_temp[test_index]
    y_train, y_validation = y_train_temp[train_index], y_train_temp[test_index]

    
x_train = np.array(x_train)
y_train = np.array(y_train)
x_validation = np.array(x_validation)
y_validation = np.array(y_validation)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
# Create user-defined function to create formatted confusion matrix
def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("\nNormalized confusion matrix")
    else:
        print('\nConfusion matrix, without normalization')

    print ()

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize = 15)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=20, fontsize = 14)
    plt.yticks(tick_marks, classes, fontsize = 14)

    plt.tight_layout()
    plt.ylabel('True label', fontsize = 14)
    plt.xlabel('Predicted label', fontsize = 14)


In [ ]:
# Run logistic regression model as a baseline
logreg_model = LogisticRegression()
logreg_model.fit(x_train, np.ravel(y_train, order='C'))
valid_predict_logreg = logreg_model.predict(x_validation)

In [ ]:
# Print classification report
print(classification_report(y_validation, valid_predict_logreg))

# Assign classes
class_names = ['First', 'Two or More']

# Compute confusion matrix
cnf_matrix_logreg = confusion_matrix(y_validation, valid_predict_logreg)
np.set_printoptions(precision=2) # set NumPy to 2 decimal places

# Plot normalized confusion matrix
plt.figure(figsize=(10,5))
plot_confusion_matrix(cnf_matrix_logreg, classes=class_names, normalize=True,
                      title='Normalized confusion matrix, Logistic Regression')

# Results: Accuracy is 68%, but very bad accuracy for identifying first admission patients

In [ ]:
# Lasso regression grid search across multiple penalties
penalties = [0.2, 0.5, 0.7, 0.9]

results_lasso = pd.DataFrame([])

for i in range(0,4):
    
    print(i)
    
    c_value = penalties[i]

    lasso_model = LogisticRegression(penalty = 'l1', C = c_value)

    lasso_model.fit(x_train, np.ravel(y_train, order='C'))

    valid_predict_lasso = lasso_model.predict(x_validation)

    class_report = pd.DataFrame(classification_report(y_validation, valid_predict_lasso, 
                                                      output_dict = True)).iloc[:, 0:2]

    class_report = class_report.T

    class_report.loc[:, 'accuracy'] = pd.DataFrame(classification_report(y_validation, valid_predict_lasso, 
                                                                         output_dict = True)).iloc[1, 2]
    class_report.loc[:, 'c'] = c_value
    class_report.loc[:, 'model'] = 'Lasso Regression'
    class_report.loc[:, 'model_value'] = ['First Treatment Episode', '2+ Treatment Episodes']

    results_lasso = pd.concat([results_lasso, class_report], axis = 0)

# Results: Results were little better than logistic regression and varied little when penalty was changed

In [ ]:
results_lasso

In [ ]:
# Ridge regression grid search across multiple penalties
penalties = [0.2, 0.5, 0.7, 0.9]

results_ridge = pd.DataFrame([])

for i in range(0,4):
    
    print(i)
    
    c_value = penalties[i]

    ridge_model = LogisticRegression(penalty = 'l2', C = c_value)

    ridge_model.fit(x_train, np.ravel(y_train, order='C'))

    valid_predict_ridge = ridge_model.predict(x_validation)

    class_report = pd.DataFrame(classification_report(y_validation, valid_predict_ridge, output_dict = True)).iloc[:, 0:2]

    class_report = class_report.T

    class_report.loc[:, 'accuracy'] = pd.DataFrame(classification_report(y_validation, valid_predict_ridge, output_dict = True)).iloc[1, 2]
    class_report.loc[:, 'c'] = c_value
    class_report.loc[:, 'model'] = 'Ridge Regression'
    class_report.loc[:, 'model_value'] = ['First Treatment Episode', '2+ Treatment Episodes']

    results_ridge = pd.concat([results_ridge, class_report], axis = 0)
    
# Results: Results were little better than logistic regression and varied little when penalty was changed

In [ ]:
ridge_results

In [ ]:
# Run default gradient boosted trees model to get baseline accuracy and feature importances
gbt_model = GradientBoostingClassifier(random_state=75)
gbt_model.fit(x_train, np.ravel(y_train, order='C'))
valid_predict_gbt = gbt_model.predict(x_validation)

In [ ]:
# Evaluation metrics and feature importances to narrow down variables I need
print(classification_report(y_validation, valid_predict_gbt))

# Print feature importances to subset data set to more important variables
print(pd.DataFrame({'features': data_imputed.columns,
                    'importances': gbt_model.feature_importances_}).sort_values(['importances'], ascending = 0))

class_names = ['First', 'Two or More']

# Compute confusion matrix
cnf_matrix_gbt = confusion_matrix(y_validation, valid_predict_gbt)
np.set_printoptions(precision=2) # set NumPy to 2 decimal places

# Plot normalized confusion matrix
plt.figure(figsize=(10,5))
plot_confusion_matrix(cnf_matrix_gbt, classes=class_names, normalize=True,
                      title='Normalized confusion matrix, Gradient Boosted Trees')

In [ ]:
# Create a medium-sized data set with all variables above 0.005 in importance
x_train_sm = x_train[:, [7, 0, 3, 45, 2, 49, 43, 26, 48, 8, 56, 41, 39, 40, 25, 37, 1, 44, 11, 4, 42, 58, 33, 30, 57, 27, 47, 28]]
x_validation_sm = x_validation[:, [7, 0, 3, 45, 2, 49, 43, 26, 48, 8, 56, 41, 39, 40, 25, 37, 1, 44, 11, 4, 42, 58, 33, 30, 57, 27, 47, 28]]
x_test_sm = x_test[:, [7, 0, 3, 45, 2, 49, 43, 26, 48, 8, 56, 41, 39, 40, 25, 37, 1, 44, 11, 4, 42, 58, 33, 30, 57, 27, 47, 28]]
features_sm = data_imputed.iloc[:, [7, 0, 3, 45, 2, 49, 43, 26, 48, 8, 56, 41, 39, 40, 25, 37, 1, 44, 11, 4, 42, 58, 33, 30, 57, 27, 47, 28]]

In [ ]:
# Ridge regression with a smaller number of variables, grid search for penalties most accurate for larger data set
penalties = [0.5, 0.9]

results_ridge_sm = pd.DataFrame([])

for i in range(0,2):
    
    print(i)
    
    c_value = penalties[i]

    ridge_model = LogisticRegression(penalty = 'l2', C = c_value)

    ridge_model.fit(x_train_sm, np.ravel(y_train, order='C'))

    valid_predict_ridge = ridge_model.predict(x_validation_sm)

    class_report = pd.DataFrame(classification_report(y_validation, valid_predict_ridge, output_dict = True)).iloc[:, 0:2]

    class_report = class_report.T

    class_report.loc[:, 'accuracy'] = pd.DataFrame(classification_report(y_validation, valid_predict_ridge, output_dict = True)).iloc[1, 2]
    class_report.loc[:, 'c'] = c_value
    class_report.loc[:, 'model'] = 'Ridge Regression'
    class_report.loc[:, 'model_value'] = ['First Treatment Episode', '2+ Treatment Episodes']

    results_ridge_sm = pd.concat([results_ridge_sm, class_report], axis = 0)
    
# Results: Accuracy dropped to 67% with smaller number of variables

In [ ]:
results_ridge_sm

In [ ]:
# Compare accuracy of default model with medium and large data frames
# Run default gradient boosted trees model to get baseline accuracy and feature importances
gbt_model_sm = GradientBoostingClassifier(random_state=75)
gbt_model_sm.fit(x_train_sm, np.ravel(y_train, order='C'))
valid_predict_gbt_sm = gbt_model_sm.predict(x_validation_sm)

In [ ]:
# Evaluation metrics are the same as with the full data set so stick with the limited data set
print(classification_report(y_validation, valid_predict_gbt_sm))

class_names = ['First', 'Two or More']

# Compute confusion matrix
cnf_matrix_gbt_sm = confusion_matrix(y_validation, valid_predict_gbt_sm)
np.set_printoptions(precision=2) # set NumPy to 2 decimal places

# Plot normalized confusion matrix
plt.figure(figsize=(10,5))
plot_confusion_matrix(cnf_matrix_gbt_sm, classes=class_names, normalize=True,
                      title='Normalized confusion matrix, Gradient Boosted Trees')

# Results: Accuracy increased to 70%, but first treatment episode accuracy did not change

In [ ]:
# Gradient boosted trees, grid search varying the number of estimators
n_estimators = [150, 200, 300]

results_gbt_nEst = pd.DataFrame([])

for i in range(0,3):
    
    print(i)
    
    est_value = n_estimators[i]

    gbt_model = GradientBoostingClassifier(random_state = 75, n_estimators = est_value)

    gbt_model.fit(x_train_sm, np.ravel(y_train, order='C'))

    valid_predict_gbt = gbt_model.predict(x_validation_sm)

    class_report = pd.DataFrame(classification_report(y_validation, valid_predict_gbt, output_dict = True)).iloc[:, 0:2]

    class_report = class_report.T

    class_report.loc[:, 'accuracy'] = pd.DataFrame(classification_report(y_validation, valid_predict_gbt, output_dict = True)).iloc[1, 2]
    class_report.loc[:, 'n_estimators'] = est_value
    class_report.loc[:, 'model'] = 'Gradient Boosted Trees'
    class_report.loc[:, 'model_value'] = ['First Treatment Episode', '2+ Treatment Episodes']

    results_gbt_nEst = pd.concat([results_gbt_nEst, class_report], axis = 0)
    
# Results: Accuracy increased with the number of estimators, but 200 and 300 were not different enough to justify the extra processing time

In [ ]:
results_gbt_nEst
# Minimal increase in accuracy for 300 and 200 instead of 150 so will stick with 150 for further testing

In [ ]:
# Gradient boosted trees, varying the maximum depth of the tree
max_depth = [5, 7, 9, 15]

results_gbt_max_depth = pd.DataFrame([])

for i in range(0,4):
    
    print(i)
    
    max_depth_value = max_depth[i]
    
    gbt_model = GradientBoostingClassifier(random_state = 75, n_estimators = 200, max_depth = max_depth_value)

    gbt_model.fit(x_train_sm, np.ravel(y_train, order='C'))

    valid_predict_gbt = gbt_model.predict(x_validation_sm)

    class_report = pd.DataFrame(classification_report(y_validation, valid_predict_gbt, 
                                                      output_dict = True)).iloc[:, 0:2]

    class_report = class_report.T

    class_report.loc[:, 'accuracy'] = pd.DataFrame(classification_report(y_validation, valid_predict_gbt, 
                                                                         output_dict = True)).iloc[1, 2]
    class_report.loc[:, 'max_depth'] = max_depth_value
    class_report.loc[:, 'n_estimators'] = 200
    class_report.loc[:, 'model'] = 'Gradient Boosted Trees'
    class_report.loc[:, 'model_value'] = ['First Treatment Episode', '2+ Treatment Episodes']

    results_gbt_max_depth = pd.concat([results_gbt_max_depth, class_report], axis = 0)
    
# Results: Accuracy increased with the depth of the tree, but did not increase enough from 7 to 9 to justify extra processing time

In [ ]:
results_gbt_max_depth

In [ ]:
# Gradient boosted trees, grid search varying the learning rate
# Going to compare the results of this to the results from n_estimators because these hyperparameters present a trade off
learning_rate = [0.05, 0.15, 0.3, 0.35]

results_gbt_learn_rate = pd.DataFrame([])

for i in range(0,4):
    
    print(i)
    
    learn_rate_value = learning_rate[i]

    gbt_model = GradientBoostingClassifier(random_state = 75, learning_rate = learn_rate_value)

    gbt_model.fit(x_train_sm, np.ravel(y_train, order='C'))

    valid_predict_gbt = gbt_model.predict(x_validation_sm)

    class_report = pd.DataFrame(classification_report(y_validation, valid_predict_gbt, 
                                                      output_dict = True)).iloc[:, 0:2]

    class_report = class_report.T

    class_report.loc[:, 'accuracy'] = pd.DataFrame(classification_report(y_validation, valid_predict_gbt, 
                                                                         output_dict = True)).iloc[1, 2]
    class_report.loc[:, 'learning_rate'] = learn_rate_value
    class_report.loc[:, 'model'] = 'Gradient Boosted Trees'
    class_report.loc[:, 'model_value'] = ['First Treatment Episode', '2+ Treatment Episodes']

    results_gbt_learn_rate = pd.concat([results_gbt_learn_rate, class_report], axis = 0)

# Results: Accuracy optimal with a learning rate of 0.3, results more accurate than with n_estimators altered

In [ ]:
results_gbt_learning_rate

In [ ]:
# Gradient boosted trees, grid search varying learning rate and maximum depth of trees
learning_rate = [0.05, 0.15, 0.3]
max_depth = [5, 7, 9]

results_gbt_lrate_depth = pd.DataFrame([])

for i, j in itertools.product(range(0,3), range(0,3)):
    
    print(i)
    print(j)
    
    max_depth_value = max_depth[i]
    
    learning_rate_value = learning_rate[j]

    gbt_model = GradientBoostingClassifier(random_state = 75, max_depth = max_depth_value,
                                           learning_rate = learning_rate_value)

    gbt_model.fit(x_train_sm, np.ravel(y_train, order='C'))

    valid_predict_gbt = gbt_model.predict(x_validation_sm)

    class_report = pd.DataFrame(classification_report(y_validation, valid_predict_gbt, 
                                                      output_dict = True)).iloc[:, 0:2]

    class_report = class_report.T

    class_report.loc[:, 'accuracy'] = pd.DataFrame(classification_report(y_validation, valid_predict_gbt, 
                                                                         output_dict = True)).iloc[1, 2]
    class_report.loc[:, 'max_depth'] = max_depth_value
    class_report.loc[:, 'learning_rate'] = learning_rate_value
    class_report.loc[:, 'model'] = 'Gradient Boosted Trees'
    class_report.loc[:, 'model_value'] = ['First Treatment Episode', '2+ Treatment Episodes']

    results_gbt_lrate_depth = pd.concat([results_gbt_lrate_depth, class_report], axis = 0)

# Results: Optimal results with max_depth = 7 and learning_rate = 0.3
# Accuracy = 0.72, F1-score for 1st treatment: 0.54, F1-score for 2+ treatments: 0.80

In [ ]:
results_gbt_lrate_depth

In [ ]:
# Gradient boosted trees, attempting early stopping with minimum impurity decrease
min_imp_dec = [0.25, 0.5, 0.1]

results_gbt_imp_dec = pd.DataFrame([])

for i in range(0,3):
    
    print(i)
    
    min_imp_value = min_imp_dec[i]

    gbt_model = GradientBoostingClassifier(random_state = 75, learning_rate = 0.3, 
                                           min_impurity_decrease = min_imp_value)

    gbt_model.fit(x_train_sm, np.ravel(y_train, order='C'))

    valid_predict_gbt = gbt_model.predict(x_validation_sm)

    class_report = pd.DataFrame(classification_report(y_validation, valid_predict_gbt, 
                                                      output_dict = True)).iloc[:, 0:2]

    class_report = class_report.T

    class_report.loc[:, 'accuracy'] = pd.DataFrame(classification_report(y_validation, valid_predict_gbt, 
                                                                         output_dict = True)).iloc[1, 2]
    class_report.loc[:, 'min_impurity_decrease'] = min_imp_value
    class_report.loc[:, 'learning_rate'] = 0.3
    class_report.loc[:, 'model'] = 'Gradient Boosted Trees'
    class_report.loc[:, 'model_value'] = ['First Treatment Episode', '2+ Treatment Episodes']

    results_gbt_imp_dec = pd.concat([results_gbt_imp_dec, class_report], axis = 0)

# Results: Accuracy decreased by several points on average

In [ ]:
results_gbt_imp_dec

In [ ]:
# Gradient boosted trees, grid search varying maximum depth and max_features
max_depth = [0.5, 0.1]
max_features = [7, 10, 15]

results_gbt_feat_maxdepth = pd.DataFrame([])

for i, j in itertools.product(range(0,2), range(0,3)):
    
    max_depth_value = max_depth[i]
    
    max_feat_value = max_features[j]

    gbt_model = GradientBoostingClassifier(random_state = 75, max_depth = max_depth_value,
                                           learning_rate = 0.3, max_features = max_feat_value)

    gbt_model.fit(x_train_sm, np.ravel(y_train, order='C'))

    valid_predict_gbt = gbt_model.predict(x_validation_sm)

    class_report = pd.DataFrame(classification_report(y_validation, valid_predict_gbt, 
                                                      output_dict = True)).iloc[:, 0:2]

    class_report = class_report.T

    class_report.loc[:, 'accuracy'] = pd.DataFrame(classification_report(y_validation, valid_predict_gbt, 
                                                                         output_dict = True)).iloc[1, 2]
    class_report.loc[:, 'max_depth'] = min_imp_dec_value
    class_report.loc[:, 'max_features'] = max_feat_value
    class_report.loc[:, 'learning_rate'] = 0.3
    class_report.loc[:, 'model'] = 'Gradient Boosted Trees'
    class_report.loc[:, 'model_value'] = ['First Treatment Episode', '2+ Treatment Episodes']

    results_gbt_feat_maxdepth = pd.concat([results_gbt_feat_maxdepth, class_report], axis = 0)

# Results: Adding a maximum feature parameter changed the model metrics very little

In [ ]:
results_gbt_feat_maxdepth

In [ ]:
# Stochastic Gradient Boosted Trees, grid search with varying sub-samples
sub_sample = [0.8, 0.9, 0.95]

results_stoch_grad = pd.DataFrame([])

for i in range(0,3):
    
    print(i)
    
    subsample_value = sub_sample[i]

    stoch_grad_model = GradientBoostingClassifier(random_state = 75, learning_rate = 0.3, max_depth = 9,
                                          subsample = subsample_value)

    stoch_grad_model.fit(x_train_sm, np.ravel(y_train, order='C'))

    valid_predict_stoch_grad = stoch_grad_model.predict(x_validation_sm)

    class_report = pd.DataFrame(classification_report(y_validation, valid_predict_stoch_grad, 
                                                      output_dict = True)).iloc[:, 0:2]

    class_report = class_report.T

    class_report.loc[:, 'accuracy'] = pd.DataFrame(classification_report(y_validation, valid_predict_gbt, 
                                                                         output_dict = True)).iloc[1, 2]
    class_report.loc[:, 'subsample'] = subsample_value
    class_report.loc[:, 'learning_rate'] = 0.3
    class_report.loc[:, 'max_depth'] = 9
    class_report.loc[:, 'model'] = 'Stochastic Gradient Boosted Trees'
    class_report.loc[:, 'model_value'] = ['First Treatment Episode', '2+ Treatment Episodes']

    results_stoch_grad = pd.concat([results_stoch_grad, class_report], axis = 0)

# Results: Adding a sub-sampling parameter changed the model metrics very little

In [ ]:
results_stoch_grad

In [ ]:
# Running a default random forest model
rf_model_default = RandomForestClassifier(random_state=75)
rf_model_default.fit(x_train_sm, np.ravel(y_train, order='C'))
valid_predict_rf_def = rf_model_default.predict(x_validation_sm)

# Results: Accuracy is 69% with F1-score of 52% for first treatment and 77% for 2+ treatments

In [ ]:
# Print classification report
print(classification_report(y_validation, valid_predict_rf_def))

class_names = ['First', 'Two or More']

# Compute confusion matrix
cnf_matrix_rf_def = confusion_matrix(y_validation, valid_predict_rf_def)
np.set_printoptions(precision=2) # set NumPy to 2 decimal places

# Plot normalized confusion matrix
plt.figure(figsize=(10,5))
plot_confusion_matrix(cnf_matrix_rf_def, classes=class_names, normalize=True,
                      title='Normalized confusion matrix, Random Forest')

In [ ]:
# Random Forest, grid search varying the number of estimators
n_estimators = [200, 300, 350]

results_rf_n_est = pd.DataFrame([])

for i in range(0,3):
    
    print(i)
    
    n_est_value = n_estimators[i]

    rf_model = RandomForestClassifier(random_state = 75, n_jobs = 7, n_estimators = n_est_value)

    rf_model.fit(x_train_sm, np.ravel(y_train, order='C'))

    valid_predict_rf = rf_model.predict(x_validation_sm)

    class_report = pd.DataFrame(classification_report(y_validation, valid_predict_rf, 
                                                      output_dict = True)).iloc[:, 0:2]

    class_report = class_report.T

    class_report.loc[:, 'accuracy'] = pd.DataFrame(classification_report(y_validation, valid_predict_rf, 
                                                                         output_dict = True)).iloc[1, 2]
    class_report.loc[:, 'n_estimators'] = n_est_value
    class_report.loc[:, 'model'] = 'Random Forest'
    class_report.loc[:, 'model_value'] = ['First Treatment Episode', '2+ Treatment Episodes']

    results_rf_n_est = pd.concat([results_rf_n_est, class_report], axis = 0)

# Results: An increase in accuracy until 300, enough to warrant keeping 300 over 200 although similar

In [ ]:
results_rf_n_est

In [ ]:
# Random Forest, grid search varying the minimum samples per split
min_samples_split = [5, 9, 12]

results_rf_min_samp = pd.DataFrame([])

for i in range(0,3):
    
    print(i)
    
    min_samp_split_value = min_samples_split[i]

    rf_model = RandomForestClassifier(random_state = 75, n_jobs = 7, n_estimators = 300,
                                     min_samples_split = min_samp_split_value)

    rf_model.fit(x_train_sm, np.ravel(y_train, order='C'))

    valid_predict_rf = rf_model.predict(x_validation_sm)

    class_report = pd.DataFrame(classification_report(y_validation, valid_predict_rf, 
                                                      output_dict = True)).iloc[:, 0:2]

    class_report = class_report.T

    class_report.loc[:, 'accuracy'] = pd.DataFrame(classification_report(y_validation, valid_predict_rf, 
                                                                         output_dict = True)).iloc[1, 2]
    class_report.loc[:, 'min_samples_split'] = min_samp_split_value
    class_report.loc[:, 'n_estimators'] = 300
    class_report.loc[:, 'model'] = 'Random Forest'
    class_report.loc[:, 'model_value'] = ['First Treatment Episode', '2+ Treatment Episodes']

    results_rf_min_samp = pd.concat([results_rf_min_samp, class_report], axis = 0)
    
# Results: Accuracy optimized at min_samples_split = 9

In [ ]:
results_rf_min_samp

In [ ]:
# Random Forest, testing early stopping with a grid search on minimum impurity decrease per split
min_impurity_decrease = [0.03, 0.05, 0.1]

results_rf_imp_dec = pd.DataFrame([])

for i in range(0,3):
    
    print(i)
    
    min_imp_dec_value = min_impurity_decrease[i]

    rf_model = RandomForestClassifier(random_state = 75, n_jobs = 7, n_estimators = 300,
                                     min_impurity_decrease = min_imp_dec_value)

    rf_model.fit(x_train_sm, np.ravel(y_train, order='C'))

    valid_predict_rf = rf_model.predict(x_validation_sm)

    class_report = pd.DataFrame(classification_report(y_validation, valid_predict_rf, 
                                                      output_dict = True)).iloc[:, 0:2]

    class_report = class_report.T

    class_report.loc[:, 'accuracy'] = pd.DataFrame(classification_report(y_validation, valid_predict_rf, 
                                                                         output_dict = True)).iloc[1, 2]
    class_report.loc[:, 'min_impurity_decrease'] = min_imp_dec_value
    class_report.loc[:, 'n_estimators'] = 300
    class_report.loc[:, 'model'] = 'Random Forest'
    class_report.loc[:, 'model_value'] = ['First Treatment Episode', '2+ Treatment Episodes']

    results_rf_imp_dec = pd.concat([results_rf_imp_dec, class_report], axis = 0)

# Results: Accuracy dropped a large amount to mid-60%s

In [ ]:
results_rf_imp_dec

In [ ]:
# Random Forest, grid search varying the maximum features and minimum samples per split
max_features = [7, 10, 15]
min_samples_split = [9, 12]

results_rf_feat_samp_split = pd.DataFrame([])

for i, j in itertools.product(range(0,3), range(0,2)):
    
    print(i)
    print(j)
    
    max_feat_value = max_features[i]
    min_samp_split_value = min_samples_split[j]

    rf_model = RandomForestClassifier(random_state = 75, n_jobs = 7, n_estimators = 300,
                                     max_features = max_feat_value, min_samples_split = min_samp_split_value)

    rf_model.fit(x_train_sm, np.ravel(y_train, order='C'))

    valid_predict_rf = rf_model.predict(x_validation_sm)

    class_report = pd.DataFrame(classification_report(y_validation, valid_predict_rf, 
                                                      output_dict = True)).iloc[:, 0:2]

    class_report = class_report.T

    class_report.loc[:, 'accuracy'] = pd.DataFrame(classification_report(y_validation, valid_predict_rf, 
                                                                         output_dict = True)).iloc[1, 2]
    class_report.loc[:, 'n_estimators'] = 300
    class_report.loc[:, 'max_features'] = max_feat_value
    class_report.loc[:, 'min_samples_split'] = min_samp_split_value
    class_report.loc[:, 'model'] = 'Random Forest'
    class_report.loc[:, 'model_value'] = ['First Treatment Episode', '2+ Treatment Episodes']

    results_rf_feat_samp_split = pd.concat([results_rf_feat_samp_split, class_report], axis = 0)

# Results: The above changes attempted did not increase accuracy above the other models

In [ ]:
results_rf_feat_samp_split

In [ ]:
# Random Forest, varying the penalty/weight put on wrong answers for 0, which is more commonly misclassified
# Also varying minimum samples per split
min_samples_split = [9, 12]
weight = [1.5, 2, 2.5, 3]

results_rf_feat_samp_split = pd.DataFrame([])

for i, j in itertools.product(range(0,2), range(0,4)):
    
    print(i)
    print(j)
    
    max_feat_value = n_estimators[i]
    weight_value = weight[j]

    rf_model = RandomForestClassifier(random_state = 75, n_jobs = 7, n_estimators = 300,
                                     class_weight = {0:weight_value}, min_samples_split = min_samp_split_value)

    rf_model.fit(x_train_sm, np.ravel(y_train, order='C'))

    valid_predict_rf = rf_model.predict(x_validation_sm)

    class_report = pd.DataFrame(classification_report(y_validation, valid_predict_rf, 
                                                      output_dict = True)).iloc[:, 0:2]

    class_report = class_report.T

    class_report.loc[:, 'accuracy'] = pd.DataFrame(classification_report(y_validation, valid_predict_rf, 
                                                                         output_dict = True)).iloc[1, 2]
    class_report.loc[:, 'n_estimators'] = 300
    class_report.loc[:, 'weight_zero'] = weight_value
    class_report.loc[:, 'min_samples_split'] = min_samp_split_value
    class_report.loc[:, 'model'] = 'Random Forest'
    class_report.loc[:, 'model_value'] = ['First Treatment Episode', '2+ Treatment Episodes']

    results_rf_feat_samp_split = pd.concat([results_rf_feat_samp_split, class_report], axis = 0)

# Results: Classification of 0 values greatly improved without sacrificing much overall accuracy
# Accuracy = 0.70, F1-score for 0 = 0.62, F1-score for 1 = 0.76

In [ ]:
results_rf_feat_samp_split

In [ ]:
# K-Nearest Neighbors, grid search varying k with Euclidean distance
k = [5, 8, 12, 15]

knn_euclidean = pd.DataFrame([])

for i in range(0,4):
    
    print(i)
    
    k_value = k[i]

    knn_model = KNeighborsClassifier(n_neighbors = k_value, p = 2, n_jobs = 7)

    knn_model.fit(x_train_sm, np.ravel(y_train, order='C'))

    valid_predict_knn = knn_model.predict(x_validation_sm)

    class_report = pd.DataFrame(classification_report(y_validation, valid_predict_knn, 
                                                      output_dict = True)).iloc[:, 0:2]

    class_report = class_report.T

    class_report.loc[:, 'accuracy'] = pd.DataFrame(classification_report(y_validation, valid_predict_knn, 
                                                                         output_dict = True)).iloc[1, 2]
    class_report.loc[:, 'k'] = k_value
    class_report.loc[:, 'distance_metric'] = 'Euclidean'
    class_report.loc[:, 'model'] = 'K-Nearest Neighbors'
    class_report.loc[:, 'model_value'] = ['First Treatment Episode', '2+ Treatment Episodes']

    knn_euclidean = pd.concat([knn_euclidean, class_report], axis = 0)

# Results: Accuracy dropped from mid to high 60s with lower F1-scores

In [ ]:
knn_euclidean

In [ ]:
# K-Nearest Neighbors, grid search varying k for Minkowski distance
k = [5, 8, 12, 15]

knn_minkowski = pd.DataFrame([])

for i in range(0,4):
    
    print(i)
    
    k_value = k[i]

    knn_model = KNeighborsClassifier(n_neighbors = k_value, n_jobs = 7)

    knn_model.fit(x_train_sm, np.ravel(y_train, order='C'))

    valid_predict_knn = knn_model.predict(x_validation_sm)

    class_report = pd.DataFrame(classification_report(y_validation, valid_predict_knn, 
                                                      output_dict = True)).iloc[:, 0:2]

    class_report = class_report.T

    class_report.loc[:, 'accuracy'] = pd.DataFrame(classification_report(y_validation, valid_predict_knn, 
                                                                         output_dict = True)).iloc[1, 2]
    class_report.loc[:, 'k'] = k_value
    class_report.loc[:, 'distance_metric'] = 'Minkowski'
    class_report.loc[:, 'model'] = 'K-Nearest Neighbors'
    class_report.loc[:, 'model_value'] = ['First Treatment Episode', '2+ Treatment Episodes']

    knn_minkowski = pd.concat([knn_minkowski, class_report], axis = 0)

# Results: Accuracy looked virtually identical to Euclidean distance

In [ ]:
knn_minkowski

In [ ]:
# Best model for each model type
## Lasso Regression
lasso_model_final = LogisticRegression(penalty = 'l1', C = 0.5)
lasso_model_final.fit(x_train, np.ravel(y_train, order='C'))
valid_predict_lasso_final = lasso_model_final.predict(x_validation)

class_names = ['First', 'Two or More']

# Compute confusion matrix
cnf_matrix_lasso = confusion_matrix(y_validation, valid_predict_lasso_final)
np.set_printoptions(precision=2) # set NumPy to 2 decimal places

# Plot normalized confusion matrix
plt.figure(figsize=(10,5))
plot_confusion_matrix(cnf_matrix_lasso, classes=class_names, normalize=True,
                      title='Normalized confusion matrix, Lasso Regression')

In [ ]:
## Ridge Regression
ridge_model_final = LogisticRegression(penalty = 'l2', C = 0.5)
ridge_model_final.fit(x_train_sm, np.ravel(y_train, order='C'))
valid_predict_ridge_final = ridge_model_final.predict(x_validation_sm)

class_names = ['First', 'Two or More']

# Compute confusion matrix
cnf_matrix_ridge = confusion_matrix(y_validation, valid_predict_ridge_final)
np.set_printoptions(precision=2) # set NumPy to 2 decimal places

# Plot normalized confusion matrix
plt.figure(figsize=(10,5))
plot_confusion_matrix(cnf_matrix_ridge, classes=class_names, normalize=True,
                      title='Normalized confusion matrix, Ridge Regression')

In [ ]:
## Gradient Boosted Trees
gbt_model_final = GradientBoostingClassifier(random_state = 75, learning_rate = 0.3, max_depth = 7)
gbt_model_final.fit(x_train_sm, np.ravel(y_train, order='C'))
valid_predict_gbt_final = gbt_model_final.predict(x_validation_sm)

class_names = ['First', 'Two or More']

# Compute confusion matrix
cnf_matrix_gbt = confusion_matrix(y_validation, valid_predict_gbt_final)
np.set_printoptions(precision=2) # set NumPy to 2 decimal places

# Plot normalized confusion matrix
plt.figure(figsize=(10,5))
plot_confusion_matrix(cnf_matrix_gbt, classes=class_names, normalize=True,
                      title='Normalized confusion matrix, Gradient Boosted Trees')

In [ ]:
## Random Forest
rf_model_final = RandomForestClassifier(random_state = 75, n_jobs = 7, n_estimators = 300,
                                        class_weight = {0:2}, min_samples_split = 12)
rf_model_final.fit(x_train_sm, np.ravel(y_train, order='C'))
valid_predict_rf_final = rf_model_final.predict(x_validation_sm)

class_names = ['First', 'Two or More']

# Compute confusion matrix
cnf_matrix_rf = confusion_matrix(y_validation, valid_predict_rf_final)
np.set_printoptions(precision=2) # set NumPy to 2 decimal places

# Plot normalized confusion matrix
plt.figure(figsize=(10,5))
plot_confusion_matrix(cnf_matrix_rf, classes=class_names, normalize=True,
                      title='Normalized confusion matrix, Random Forest')

In [ ]:
## K-Nearest Neighbors
knn_model_final = KNeighborsClassifier(n_neighbors = 12, n_jobs = 7)
knn_model_final.fit(x_train_sm, np.ravel(y_train, order='C'))
valid_predict_knn_final = knn_model_final.predict(x_validation_sm)

class_names = ['First', 'Two or More']

# Compute confusion matrix
cnf_matrix_knn = confusion_matrix(y_validation, valid_predict_knn_final)
np.set_printoptions(precision=2) # set NumPy to 2 decimal places

# Plot normalized confusion matrix
plt.figure(figsize=(10,5))
plot_confusion_matrix(cnf_matrix_knn, classes=class_names, normalize=True,
                      title='Normalized confusion matrix, Random Forest')

In [ ]:
# Building and printing ROC curves for all final models
classes = ['First', 'Two or More']

probs1 = lasso_model_final.predict_proba(x_validation)
probs1 = probs1[:, 1]

probs2 = rf_model_final.predict_proba(x_validation_sm)
probs2 = probs2[:, 1]

probs3 = gbt_model_final.predict_proba(x_validation_sm)
probs3 = probs3[:, 1]

probs4 = logreg_model.predict_proba(x_validation)
probs4 = probs4[:, 1]

probs5 = knn_model_final.predict_proba(x_validation_sm)
probs5 = probs5[:, 1]

probs6 = ridge_model_final.predict_proba(x_validation_sm)
probs6 = probs6[:, 1]

fpr, tpr, thresh = metrics.roc_curve(y_validation, probs3)
auc = metrics.roc_auc_score(y_validation, probs3)
np.set_printoptions(precision=2)
plt.plot(fpr,tpr,label="Gradient Boosted Trees, auc="+str(round(auc, 2)), color = 'red')

fpr, tpr, thresh = metrics.roc_curve(y_validation, probs2)
auc = metrics.roc_auc_score(y_validation, probs2)
np.set_printoptions(precision=2)
plt.plot(fpr,tpr,label="Random Forest, auc="+str(round(auc, 2)), color = 'blue')

fpr, tpr, thresh = metrics.roc_curve(y_validation, probs5)
auc = metrics.roc_auc_score(y_validation, probs5)
np.set_printoptions(precision=2)
plt.plot(fpr,tpr,label="K-Nearest Neighbors, auc="+str(round(auc, 2)), color = 'darkorchid')

fpr, tpr, thresh = metrics.roc_curve(y_validation, probs4)
auc = metrics.roc_auc_score(y_validation, probs4)
np.set_printoptions(precision=2)
plt.plot(fpr,tpr,label="Logistic Regression, auc="+str(round(auc, 2)), color = 'gold')

fpr, tpr, thresh = metrics.roc_curve(y_validation, probs1)
auc = metrics.roc_auc_score(y_validation, probs1)
np.set_printoptions(precision=2)
plt.plot(fpr,tpr,label="Lasso Regression, auc="+str(round(auc, 2)), color = 'darkorange')

fpr, tpr, thresh = metrics.roc_curve(y_validation, probs6)
auc = metrics.roc_auc_score(y_validation, probs6)
np.set_printoptions(precision=2)
plt.plot(fpr,tpr,label="Ridge Regression, auc="+str(round(auc, 2)), color = 'green')

plt.legend(loc=0, fontsize = 'x-large')
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize = 35)
plt.xticks(fontsize = 25)
plt.yticks(fontsize = 25)
plt.xlabel('False Positive Rate', fontsize = 30)
plt.ylabel('True Positive Rate', fontsize = 30)
plt.plot([0, 1], [0, 1], color='black', linestyle='--')

fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 20
fig_size[1] = 15
plt.rcParams["figure.figsize"] = fig_size

plt.show()

plt.savefig('NumPriorTreat_ROC_Curve_Comp.png')

In [ ]:
# Model Chosen: Random Forest with min_samples_split = 12, n_estimators = 300, and a weight of 2 on misclassifications of zero
# Above model shows lower accuracy on the ROC curve than gradient boosted trees but is the only model that classified 0s significantly higher than chance
# Testing model with final holdout data set
valid_predict_rf_ho = rf_model_final.predict(x_test_sm)

class_names = ['First', 'Two or More']

# Compute confusion matrix
cnf_matrix_rf_ho = confusion_matrix(y_test, valid_predict_rf_ho)
np.set_printoptions(precision=2) # set NumPy to 2 decimal places

# Plot normalized confusion matrix
plt.figure(figsize=(10,5))
plot_confusion_matrix(cnf_matrix_rf, classes=class_names, normalize=True,
                      title='Normalized Confusion Matrix, Random Forest, Holdout Data')

pd.DataFrame(classification_report(y_test, valid_predict_rf_ho, 
                                   output_dict = True))